In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import re
from konlpy.tag import Okt
from tqdm import tqdm
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import os
from sklearn.model_selection import train_test_split
from collections import Counter

In [25]:
import glob

In [2]:
# pip install konlpy

     ---------------------------------------- 19.4/19.4 MB 1.3 MB/s eta 0:00:00
     ---------------------------------------- 3.6/3.6 MB 2.7 MB/s eta 0:00:00
     -------------------------------------- 362.6/362.6 KB 2.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [4]:
# pip install tensorflow

     -------------------------------------- 438.0/438.0 MB 1.0 MB/s eta 0:00:00
     -------------------------------------- 57.5/57.5 KB 605.5 kB/s eta 0:00:00
     ---------------------------------------- 1.4/1.4 MB 1.3 MB/s eta 0:00:00
     ---------------------------------------- 1.5/1.5 MB 1.2 MB/s eta 0:00:00
     -------------------------------------- 462.5/462.5 KB 1.5 MB/s eta 0:00:00
     ---------------------------------------- 5.8/5.8 MB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     -------------------------------------- 126.7/126.7 KB 1.8 MB/s eta 0:00:00
     ---------------------------------------- 42.6/42.6 KB 1.0 MB/s eta 0:00:00
     -------------------------------------- 895.9/895.9 KB 2.2 MB/s eta 0:00:00
     ---------------------------------------- 3.4/3.4 MB 1.6 MB/s eta 0:00:00
     ---------------------------------------- 13.9/13.9 MB 1.4 MB/s eta 0:00:00
     ---------------------

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.



     -------------------------------------- 781.3/781.3 KB 2.1 MB/s eta 0:00:00
     ---------------------------------------- 97.8/97.8 KB 1.9 MB/s eta 0:00:00
     -------------------------------------- 289.2/289.2 KB 2.0 MB/s eta 0:00:00
     -------------------------------------- 156.5/156.5 KB 1.9 MB/s eta 0:00:00
  Using cached pyasn1_modules-0.2.8-py2.py3-none-any.whl (155 kB)
  Using cached pyasn1-0.4.8-py2.py3-none-any.whl (77 kB)
     -------------------------------------- 151.5/151.5 KB 1.8 MB/s eta 0:00:00
Using legacy 'setup.py install' for termcolor, since package 'wheel' is not installed.
  Running setup.py install for termcolor: started
  Running setup.py install for termcolor: finished with status 'done'


In [6]:
# pip install sklearn

  Using cached sklearn-0.0.tar.gz (1.1 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 7.2/7.2 MB 2.4 MB/s eta 0:00:00
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1316 sha256=0eb2a1f80786b14e7f4fe93eb9a238208c82bd2c9b4560e3f8c5fff97099967f
  Stored in directory: c:\users\easya\appdata\local\pip\cache\wheels\22\0b\40\fd3f795caaa1fb4c6cb738bc1f56100be1e57da95849bfc897
Successfully built sklearn
Note: you may need to restart the kernel to use updated packages.


## 1. 데이터 불러오기

In [8]:
# 현재 작업 경로 확인
print(os.getcwd())

c:\Users\easya\Workspace\HK_Project\7_자연어처리


### 1-1. "반도체"가 제목에 포함된 네이버 증권 뉴스 데이터

In [10]:
# newstitle 데이터 확인
newstitle = pd.read_csv("../data/Newsfile3.csv", encoding='euc-kr')
newstitle.tail(3)

,date,title
58439,20151019,"[특징주]제주반도체, 美·中 대규모 투자유치에 '上'"
58440,20151019,[반도체] 2016 반도체 투자 감소 예상 - 미래
58441,20151019,"제주반도체, '해외투자유치설' 조회공시 요구"


### 1-2. 휴장일 데이터

In [20]:
# 2015~2016 휴장일 데이터 확인 ()
file1 = pd.read_excel("../data/closedate_2015.xls")
file2 = pd.read_excel("../data/closedate_2016.xls")

In [21]:
# file1 데이터 형식 확인
file1.head(2)

,일자 및 요일,요일구분,비고
0,2015-01-01,목요일,신정
1,2015-02-18,수요일,설날


In [22]:
# file2 데이터 형식 확인
file2.head(2)

,일자 및 요일,요일구분,비고
0,2016-01-01,금요일,신정
1,2016-02-08,월요일,설날


In [43]:
# ./data 에 있는 폴더와 파일경로 확인
files = glob.glob('../data/closedate_*.xls')
files

['../data\\closedate_2015.xls',
 '../data\\closedate_2016.xls',
 '../data\\closedate_2017.xls',
 '../data\\closedate_2018.xls',
 '../data\\closedate_2019.xls',
 '../data\\closedate_2020.xls',
 '../data\\closedate_2021.xls',
 '../data\\closedate_2022.xls']

In [46]:
# files의 첫번째 file만 추출
file = files[0]
file

'../data\\closedate_2015.xls'

In [48]:
# file의 제목만 추출
file.split('\\')[-1]

'closedate_2015.xls'

In [ ]:
for file in glob.glob(path, '*.xls'):
    with open(file, encoding='euc-kr') as f:
        data

In [ ]:
# 

In [13]:
# closedate = pd.read_excel("../data/closedate_2015.xls")
# closedate

,일자 및 요일,요일구분,비고
0,2015-01-01,목요일,신정
1,2015-02-18,수요일,설날
2,2015-02-19,목요일,설날
3,2015-02-20,금요일,설날
4,2015-05-01,금요일,근로자의날
5,2015-05-05,화요일,어린이날
6,2015-05-25,월요일,석가탄신일
7,2015-08-14,금요일,광복70주년 임시공휴일
8,2015-09-28,월요일,추석
9,2015-09-29,화요일,추석(대체휴일제)


## 2. 데이터 확인

In [11]:
newstitle.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58442 entries, 0 to 58441
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    58442 non-null  int64 
 1   title   58442 non-null  object
dtypes: int64(1), object(1)
memory usage: 913.3+ KB


In [10]:
# 결측치 확인
newstitle.isnull().sum()

date     0
title    0
dtype: int64